In [1]:
from imports import *
from FeatureExtraction import *
from PreProcessor import *

## Preprocessor

In [2]:
_PreProcessor = PreProcessor()
UC_documents, code_documents, UCTokens, CodeTokens = _PreProcessor.setup(
            "./Dataset/UC", "./Dataset/CC"
        )
UCTokens.update(CodeTokens)


## Initialize Normalizer

In [3]:
normalizer = MinMaxScaler(copy=False)

## Adjuting the csv file

In [4]:
DataSet = pd.read_csv("Dataset/eTour.csv", names=['UC', 'CC'])
temp = set()
DataSet['Labels'] = np.ones(len(DataSet), np.uint16)
for row_num,row in enumerate(DataSet.itertuples(index=False, name=None)):
    filename_CC_base, _ = os.path.splitext(row[1].lower())
    filename_CC = filename_CC_base + '.java'
    temp.add((row[0].lower(), filename_CC))
    DataSet.loc[row_num] = [row[0].lower(), filename_CC, 1]

for filename_UC in os.listdir("./Dataset/UC"):
    for filename_CC in os.listdir("./Dataset/CC"):
        if (filename_UC.lower(), filename_CC.lower()) not in temp:
            DataSet.loc[len(DataSet)] = [filename_UC, filename_CC, 0]

DataSet.to_csv('Dataset/eTourModified.csv', index = False)    
print(DataSet)

           UC                                 CC  Labels
0     uc1.txt  gestionebeniculturaliagenzia.java       1
1     uc1.txt               dbbeneculturale.java       1
2     uc1.txt              idbbeneculturale.java       1
3     uc1.txt                 beniculturali.java       1
4     uc2.txt               dbbeneculturale.java       1
...       ...                                ...     ...
6897  UC9.TXT                           Tag.java       0
6898  UC9.TXT                 TAGTableModel.java       0
6899  UC9.TXT         TestoNewsRendererTest.java       0
6900  UC9.TXT                       Turisti.java       0
6901  UC9.TXT             TuristiTableModel.java       0

[6902 rows x 3 columns]


## Reading the dataset

In [5]:
DataSet = pd.read_csv("Dataset/eTourModified.csv")
# DataSet)
##### incorrect data / duplicated data / NA 
# DataSet.dropna(inplace=True)
# DataSet.duplicated().to_string())

###### TO DO: Add the rest of the negative class

for row in DataSet.index:
    UC = DataSet.loc[row, 'UC'].lower()
    CC = DataSet.loc[row, 'CC'].lower()
    if UC in _PreProcessor.UC_to_index:
        DataSet.loc[row, 'UC'] = _PreProcessor.UC_to_index[UC]
    if CC in _PreProcessor.CC_to_index:
        # print(CC)
        DataSet.loc[row, 'CC'] = _PreProcessor.CC_to_index[CC]

print(DataSet.to_string())

      UC   CC  Labels
0      0   55       1
1      0   29       1
2      0   67       1
3      0   20       1
4     11   29       1
5     11   84       1
6     11   55       1
7     11   22       1
8     11    6       1
9     11   20       1
10    22   29       1
11    22   84       1
12    22   55       1
13    22   22       1
14    22    6       1
15    22   20       1
16    33   29       1
17    33   84       1
18    33   55       1
19    33  106       1
20    33  108       1
21    33  107       1
22    33    6       1
23    33   92       1
24    33   20       1
25    44    6       1
26    44   85       1
27    44   56       1
28    44   20       1
29    54   40       1
30    54   57       1
31    54   86       1
32    55   68       1
33    55   57       1
34    55   86       1
35    55    7       1
36    56   40       1
37    56   57       1
38    56   86       1
39    56   23       1
40    56   15       1
41    57   40       1
42    57   57       1
43    57   86       1
44    57  

## Collecting the 131 features

In [6]:
featureExtraction = FeatureExtraction(UCTokens)

###### used for all ######
tfidf_matrix_uc, tfidf_matrix_code,idf_uc_dict,idf_code_dict,feature_names_uc,feature_names_code ,df_uc_dict,df_code_dict= featureExtraction.TFIDFVectorizer(UC_documents, code_documents)
UC_count_matrix, code_count_matrix,tf_uc_dict,tf_code_dict = featureExtraction.CountVectorizerModel(UC_documents, code_documents, 'train')
idf_uc_q,idf_code_q= featureExtraction.IDFPreProcessing(UC_documents,idf_code_dict,code_documents,idf_uc_dict)
ictf_uc_q,ictf_code_q=featureExtraction.ICTFPreProcessing(UC_documents,tf_code_dict,code_documents,tf_uc_dict)
entropy_uc,entropy_code=featureExtraction.EntropyPreProcessing(UC_documents,code_documents,df_uc_dict,df_code_dict)
variance_uc,variance_code= featureExtraction.VarPreProcessing(UC_documents,code_documents,idf_uc_dict,idf_code_dict)
PMI_uc,PMI_code=featureExtraction.PMIPreProcessing(code_documents,UC_documents)
SCQ_uc,SCQ_code = featureExtraction.SCQPreProcessing(UC_documents,code_documents,tf_uc_dict,tf_code_dict,idf_uc_dict,idf_code_dict)

# the values of the count matrices are normalized

In [8]:
#-------------------------14 IR based features--------------------------#

# 1) Vector space model
cosine_similarities_feature = featureExtraction.VectorSpaceModel(tfidf_matrix_uc, tfidf_matrix_code)
print("cosine_similarities_feature", cosine_similarities_feature.shape)

# Our model will classify based on the ranks of the data rather than their actual values.
cosine_similarity_UC = rankdata(-cosine_similarities_feature, method='dense', axis=1)
print("cosine_similarity_UC", cosine_similarity_UC.shape)
cosine_similarity_CC = rankdata(-cosine_similarities_feature, method='dense', axis=0)
print("cosine_similarity_CC", cosine_similarity_CC.shape)

# 2) Latent semantic analysis
LSA_similraities_feature = featureExtraction.LSA(tfidf_matrix_uc, tfidf_matrix_code)
print('LSA similarity', LSA_similraities_feature.shape)

LSA_similraities_UC = rankdata(-LSA_similraities_feature, method='dense', axis=1)
print("LSA_similraities_UC", LSA_similraities_UC.shape)
LSA_similraities_CC = rankdata(-LSA_similraities_feature, method='dense', axis=0)
print("LSA_similraities_CC", LSA_similraities_CC.shape)

# 3) Latent Dirichlet Allocation
DocumentTopicDisUC_dense, DocumentTopicDisCode_dense, cosine_similarities_LDA= featureExtraction.LDA(UC_documents,code_documents,UCTokens)
print('LDA similarity', cosine_similarities_LDA.shape)

LDA_similraities_UC = rankdata(-cosine_similarities_LDA, method='dense', axis=1)
print("LDA_similraities_UC", LDA_similraities_UC.shape)
LDA_similraities_CC = rankdata(-cosine_similarities_LDA, 'dense', axis=0)
print("LDA_similraities_CC", LDA_similraities_CC.shape)

# 4) Jensen-Shannon(JS)
JS_features = featureExtraction.JensenShannon(UC_count_matrix, code_count_matrix)
print('JS', JS_features.shape)

JS_UC = rankdata(-JS_features, method='dense', axis=1)
print('JS_UC', JS_UC.shape)
JS_CC = rankdata(-JS_features, method='dense', axis=0)
print('JS_CC', JS_CC.shape)

# 5)  Okapi BM25
# UC means query is UC and document is code
BM25_UC = featureExtraction.BM25(UC_documents,code_documents,idf_code_dict,code_count_matrix)
BM25_CC = featureExtraction.BM25(code_documents,UC_documents,idf_uc_dict,UC_count_matrix)

BM25_UC = rankdata(-BM25_UC, method='dense', axis=0)
print("BM25_UC", BM25_UC.shape)
BM25_CC = rankdata(-BM25_CC, method='dense', axis=0)
print("BM25_CC", BM25_CC.shape)

# 6) Language Model with Dirichlet
# UC means UC is query and code is doc
JM_UC = featureExtraction.SmoothingMethods(UC_documents,code_documents,code_count_matrix,tf_code_dict,JM_or_DP=True)
JM_CC = featureExtraction.SmoothingMethods(code_documents,UC_documents,UC_count_matrix,tf_uc_dict,True)

DP_UC = featureExtraction.SmoothingMethods(UC_documents,code_documents,code_count_matrix,tf_code_dict,JM_or_DP=False)
DP_CC = featureExtraction.SmoothingMethods(code_documents,UC_documents,UC_count_matrix,tf_uc_dict,False)

JM_UC = rankdata(-JM_UC, method='dense', axis=0)
print("JM_UC", JM_UC.shape)
JM_CC = rankdata(-JM_CC, method='dense', axis=0)
print("JM_CC", JM_CC.shape)
DP_UC = rankdata(-DP_UC, method='dense', axis=0)
print("DP_UC", DP_UC.shape)
DP_CC = rankdata(-DP_CC, method='dense', axis=0)
print("DP_CC", DP_CC.shape)

# ------------------------pre-retrieval (21 metrics)--------------------------#
# 1) IDF Features
avg_idf_uc= featureExtraction.AvgIDF(idf_uc_q).reshape(1,-1) 
avg_idf_code= featureExtraction.AvgIDF(idf_code_q).reshape(1,-1) 

normalizer.fit_transform(avg_idf_uc)
normalizer.fit_transform(avg_idf_code)

print('avg_idf_uc_shape', avg_idf_uc.shape) 
print('avg_idf_code_shape', avg_idf_code.shape) 

max_idf_uc= featureExtraction.MaxIDF(idf_uc_q).reshape(1,-1) 
max_idf_code= featureExtraction.MaxIDF(idf_code_q).reshape(1,-1) 

normalizer.fit_transform(max_idf_uc)
normalizer.fit_transform(max_idf_code)

print('max_idf_uc_shape', max_idf_uc.shape) 
print('max_idf_code_shape', max_idf_code.shape) 

dev_idf_uc= featureExtraction.DevIDF(idf_uc_q).reshape(1,-1) 
dev_idf_code= featureExtraction.DevIDF(idf_code_q).reshape(1,-1) 

normalizer.fit_transform(dev_idf_uc)
normalizer.fit_transform(dev_idf_code)

print('dev_idf_uc_shape', dev_idf_uc.shape) 
print('dev_idf_code_shape', dev_idf_code.shape) 

# 2) ICTF Features
avg_ictf_uc= featureExtraction.AvgICTF(ictf_uc_q).reshape(1,-1) 
avg_ictf_code= featureExtraction.AvgICTF(ictf_code_q).reshape(1,-1) 

normalizer.fit_transform(avg_ictf_uc)
normalizer.fit_transform(avg_ictf_code)

print('avg_ictf_uc_shape', avg_ictf_uc.shape)
print('avg_ictf_code_shape', avg_ictf_code.shape)

max_ictf_uc= featureExtraction.MaxICTF(ictf_uc_q).reshape(1,-1) 
max_ictf_code= featureExtraction.MaxICTF(ictf_code_q).reshape(1,-1) 

normalizer.fit_transform(max_ictf_uc)
normalizer.fit_transform(max_ictf_code)

print('max_ictf_uc_shape', max_ictf_uc.shape)
print('max_ictf_code_shape', max_ictf_code.shape)

dev_ictf_uc= featureExtraction.DevICTF(ictf_uc_q).reshape(1,-1) 
dev_ictf_code= featureExtraction.DevICTF(ictf_code_q).reshape(1,-1) 

normalizer.fit_transform(dev_ictf_uc)
normalizer.fit_transform(dev_ictf_code)

print('dev_ictf_uc_shape', dev_ictf_uc.shape)
print('dev_ictf_code_shape', dev_ictf_code.shape)
# 3) Entropy Features

avg_entropy_uc= featureExtraction.AvgEntropy(entropy_uc).reshape(1,-1) 
avg_entropy_code= featureExtraction.AvgEntropy(entropy_code).reshape(1,-1) 

normalizer.fit_transform(avg_entropy_uc)
normalizer.fit_transform(avg_entropy_code)

print('avg_entropy_uc_shape', avg_entropy_uc.shape)
print('avg_entropy_code_shape', avg_entropy_code.shape)

max_entropy_uc= featureExtraction.MaxEntropy(entropy_uc).reshape(1,-1) 
max_entropy_code= featureExtraction.MaxEntropy(entropy_code).reshape(1,-1) 

normalizer.fit_transform(max_entropy_uc)
normalizer.fit_transform(max_entropy_code)

print('max_entropy_uc_shape', max_entropy_uc.shape)
print('max_entropy_code_shape', max_entropy_code.shape)

med_entropy_uc= featureExtraction.MedEntropy(entropy_uc).reshape(1,-1) 
med_entropy_code= featureExtraction.MedEntropy(entropy_code).reshape(1,-1) 

normalizer.fit_transform(med_entropy_uc)
normalizer.fit_transform(med_entropy_code)

print('med_entropy_uc_shape', med_entropy_uc.shape)
print('med_entropy_code_shape', med_entropy_code.shape)

dev_entropy_uc= featureExtraction.DevEntropy(entropy_uc).reshape(1,-1) 
dev_entropy_code= featureExtraction.DevEntropy(entropy_code).reshape(1,-1) 

normalizer.fit_transform(dev_entropy_uc)
normalizer.fit_transform(dev_entropy_code)

print('dev_entropy_uc_shape', dev_entropy_uc.shape)
print('dev_entropy_code_shape', dev_entropy_code.shape)
      

# 4) Variance Features

avg_variance_uc= featureExtraction.AvgVariance(variance_uc).reshape(1,-1) 
avg_variance_code= featureExtraction.AvgVariance(variance_code).reshape(1,-1) 

normalizer.fit_transform(avg_variance_uc)
normalizer.fit_transform(avg_variance_code)

print('avg_variance_uc_shape', avg_variance_uc.shape)
print('avg_variance_code_shape', avg_variance_code.shape)

max_variance_uc= featureExtraction.MaxVariance(variance_uc).reshape(1,-1) 
max_variance_code= featureExtraction.MaxVariance(variance_code).reshape(1,-1) 

normalizer.fit_transform(max_variance_uc)
normalizer.fit_transform(max_variance_code)

print('max_variance_uc_shape', max_variance_uc.shape)
print('max_variance_code_shape', max_variance_code.shape)

sum_variance_uc= featureExtraction.SumVariance(variance_uc).reshape(1,-1) 
sum_variance_code= featureExtraction.SumVariance(variance_code).reshape(1,-1) 

normalizer.fit_transform(sum_variance_uc)
normalizer.fit_transform(sum_variance_code)

print('sum_variance_uc_shape', sum_variance_uc.shape)
print('sum_variance_code_shape', sum_variance_code.shape)
# 5) SCQ Features
avg_scq_uc= featureExtraction.AvgSCQ(SCQ_uc).reshape(1,-1) 
avg_scq_code= featureExtraction.AvgSCQ(SCQ_code).reshape(1,-1) 

normalizer.fit_transform(avg_scq_uc)
normalizer.fit_transform(avg_scq_code)

print('avg_scq_uc_shape', avg_scq_uc.shape)
print('avg_scq_code_shape', avg_scq_code.shape)

max_scq_uc= featureExtraction.MaxSCQ(SCQ_uc).reshape(1,-1) 
max_scq_code= featureExtraction.MaxSCQ(SCQ_code).reshape(1,-1) 

normalizer.fit_transform(max_scq_uc)
normalizer.fit_transform(max_scq_code)

print('max_scq_uc_shape', max_scq_uc.shape)
print('max_scq_code_shape', max_scq_code.shape)

sum_sqc_uc= featureExtraction.SumSCQ(SCQ_uc).reshape(1,-1) 
sum_sqc_code= featureExtraction.SumSCQ(SCQ_code).reshape(1,-1) 

normalizer.fit_transform(sum_sqc_uc)
normalizer.fit_transform(sum_sqc_code)

print('sum_sqc_uc_shape', sum_sqc_uc.shape)
print('sum_sqc_code_shape', sum_sqc_code.shape)

# 6) PMI Features

avg_pmi_uc= featureExtraction.AvgPMI(PMI_uc).reshape(1,-1) 
avg_pmi_code= featureExtraction.AvgPMI(PMI_code).reshape(1,-1) 

normalizer.fit_transform(avg_pmi_uc)
normalizer.fit_transform(avg_pmi_code)

print('avg_pmi_uc_shape', avg_pmi_uc.shape)
print('avg_pmi_code_shape', avg_pmi_code.shape)

max_pmi_uc= featureExtraction.MaxPMI(PMI_uc).reshape(1,-1) 
max_pmi_code= featureExtraction.MaxPMI(PMI_code).reshape(1,-1) 

normalizer.fit_transform(max_pmi_uc)
normalizer.fit_transform(max_pmi_code)

print('max_pmi_uc_shape', max_pmi_uc.shape)
print('max_pmi_code_shape', max_pmi_code.shape)
# 7) QS Features

qs_uc= featureExtraction.QS(UC_documents,code_documents).reshape(1,-1) 
qs_code = featureExtraction.QS(code_documents,UC_documents).reshape(1,-1) 

normalizer.fit_transform(qs_uc)
normalizer.fit_transform(qs_code)

print('qs_uc_shape', qs_uc.shape)
print('qs_code_shape', qs_code.shape)

# 8) SCS Features
UC_SCS = featureExtraction.simplifiedClarityScore(UC_documents,UC_count_matrix,tf_code_dict).reshape(1,-1) 
CC_SCS = featureExtraction.simplifiedClarityScore(code_documents,code_count_matrix,tf_uc_dict).reshape(1,-1) 

normalizer.fit_transform(UC_SCS)
normalizer.fit_transform(CC_SCS)

print('UC_SCS_shape', UC_SCS.shape)
print('CC_SCS_shape', CC_SCS.shape)

# 9) Coherence Score Features
UC_CoherenceScore = featureExtraction.CoherenceScore(UC_documents,tfidf_matrix_code).reshape(1,-1) 
CC_CoherenceScore = featureExtraction.CoherenceScore(code_documents,tfidf_matrix_uc).reshape(1,-1) 

normalizer.fit_transform(UC_CoherenceScore)
normalizer.fit_transform(CC_CoherenceScore)

print('UC_CoherenceScore_shape', UC_CoherenceScore.shape)
print('CC_CoherenceScore_shape', CC_CoherenceScore.shape)

cosine_similarities_feature (58, 119)
cosine_similarity_UC (58, 119)
cosine_similarity_CC (58, 119)
LSA similarity (58, 119)
LSA_similraities_UC (58, 119)
LSA_similraities_CC (58, 119)
LDA similarity (58, 119)
LDA_similraities_UC (58, 119)
LDA_similraities_CC (58, 119)
JS (58, 119)
JS_UC (58, 119)
JS_CC (58, 119)
BM25_UC (119, 58)
BM25_CC (58, 119)
JM_UC (119, 58)
JM_CC (58, 119)
DP_UC (119, 58)
DP_CC (58, 119)
avg_idf_uc_shape (1, 58)
avg_idf_code_shape (1, 119)
max_idf_uc_shape (1, 58)
max_idf_code_shape (1, 119)
dev_idf_uc_shape (1, 58)
dev_idf_code_shape (1, 119)
avg_ictf_uc_shape (1, 58)
avg_ictf_code_shape (1, 119)
max_ictf_uc_shape (1, 58)
max_ictf_code_shape (1, 119)
dev_ictf_uc_shape (1, 58)
dev_ictf_code_shape (1, 119)
avg_entropy_uc_shape (1, 58)
avg_entropy_code_shape (1, 119)
max_entropy_uc_shape (1, 58)
max_entropy_code_shape (1, 119)
med_entropy_uc_shape (1, 58)
med_entropy_code_shape (1, 119)
dev_entropy_uc_shape (1, 58)
dev_entropy_code_shape (1, 119)
avg_variance_uc_s

### Post Retrieval features

In [ ]:
#------------------------post-retrieval (7 metrics)--------------------------#
# 1.1) Subquery overlap using jensenShannon
code_queries_score_JensenShannon = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.JensenShannon,"JS", UC_count_matrix, code_count_matrix)
UC_queries_score_JensenShannon = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.JensenShannon,"JS",code_count_matrix, UC_count_matrix)

print('code_queries_score_JensenShannon', code_queries_score_JensenShannon.shape)
print('UC_queries_score_JensenShannon', UC_queries_score_JensenShannon.shape)

# 1.2) Subquery overlap using VSM
code_queries_score_VSM = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.VectorSpaceModel,"VSM", UC_count_matrix, code_count_matrix)
UC_queries_score_VSM = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.VectorSpaceModel,"VSM", code_count_matrix, UC_count_matrix)

print('code_queries_score_VSM', code_queries_score_VSM.shape)
print('UC_queries_score_VSM', UC_queries_score_VSM.shape)

#1.3) Subquery overlap using BM25
code_queries_score_BM25 = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.BM25,"BM", UC_documents,idf_uc_dict,UC_count_matrix)
UC_queries_score_BM25 = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.BM25,"BM", code_documents,idf_code_dict, code_count_matrix)

print('code_queries_score_BM25', code_queries_score_BM25.shape)
print('UC_queries_score_BM25', UC_queries_score_BM25.shape)

#1.4) Subquery overlap using JM smoothing
code_queries_score_JM = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.SmoothingMethods,"SM", UC_documents,UC_count_matrix,tf_uc_dict,True)
UC_queries_score_JM = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.SmoothingMethods,"SM", code_documents,code_count_matrix,tf_code_dict,True)

print('code_queries_score_JM', code_queries_score_JM.shape)
print('UC_queries_score_JM', UC_queries_score_JM.shape)

# 1.5) Subquery overlap using DP smoothing
code_queries_score_DP = featureExtraction.SubqueryOverlap(code_documents, featureExtraction.SmoothingMethods,"SM", UC_documents, UC_count_matrix,tf_uc_dict,False)
UC_queries_score_DP = featureExtraction.SubqueryOverlap(UC_documents, featureExtraction.SmoothingMethods,"SM", code_documents, code_count_matrix,tf_code_dict,False)

print('code_queries_score_DP', code_queries_score_DP.shape)
print('UC_queries_score_DP', UC_queries_score_DP.shape)


In [ ]:
# 2.1) Robustness Score using jensenShannon
UC_RS_JS = featureExtraction.RobustnessScore(UC_documents,code_documents,"JS")
code_RS_JS = featureExtraction.RobustnessScore(code_documents,UC_documents,"JS")

print('UC_RS_JS', UC_RS_JS.shape)
print('code_RS_JS', code_RS_JS.shape)

# 2.2) Robustness Score using VSM
UC_RS_VSM = featureExtraction.RobustnessScore(UC_documents,code_documents,"VSM")
code_RS_VSM = featureExtraction.RobustnessScore(code_documents,UC_documents,"VSM")

print('UC_RS_VSM', UC_RS_VSM.shape)
print('code_RS_VSM', code_RS_VSM.shape)

# 2.3) Robustness Score using BM25
UC_RS_BM25 = featureExtraction.RobustnessScore(UC_documents,code_documents,"BM")
code_RS_BM25 = featureExtraction.RobustnessScore(code_documents,UC_documents,"BM")

print('UC_RS_BM25', UC_RS_BM25.shape)
print('code_RS_BM25', code_RS_BM25.shape)

# 2.4) Robustness Score using JM smoothing
UC_RS_JM = featureExtraction.RobustnessScore(UC_documents,code_documents,"JM")
code_RS_JM = featureExtraction.RobustnessScore(code_documents,UC_documents,"JM")

print('UC_RS_JM', UC_RS_JM.shape)
print('code_RS_JM', code_RS_JM.shape)

# 2.5) Robustness Score using DP smoothing
UC_RS_DP = featureExtraction.RobustnessScore(UC_documents,code_documents,"DP")
code_RS_DP = featureExtraction.RobustnessScore(code_documents,UC_documents,"DP")

print('UC_RS_DP', UC_RS_DP.shape)
print('code_RS_DP', code_RS_DP.shape)

In [ ]:
# 3.1) First Rank Change using JensenShannon
UC_FRC_JS = featureExtraction.RobustnessScore(UC_documents,code_documents,"JS",False)
code_FRC_JS = featureExtraction.RobustnessScore(code_documents,UC_documents,"JS",False)

print('UC_FRC_JS', UC_FRC_JS.shape)
print('code_FRC_JS', code_FRC_JS.shape)

# 2.2) First Rank Change using VSM
UC_FRC_VSM = featureExtraction.RobustnessScore(UC_documents,code_documents,"VSM",False)
code_FRC_VSM = featureExtraction.RobustnessScore(code_documents,UC_documents,"VSM",False)

print('UC_FRC_VSM', UC_FRC_VSM.shape)
print('code_FRC_VSM', code_FRC_VSM.shape)

# 2.3) First Rank Change using BM25
UC_FRC_BM25 = featureExtraction.RobustnessScore(UC_documents,code_documents,"BM",False)
code_FRC_BM25 = featureExtraction.RobustnessScore(code_documents,UC_documents,"BM",False)

print('UC_FRC_BM25', UC_FRC_BM25.shape)
print('code_FRC_BM25', code_FRC_BM25.shape)

# 2.4) First Rank Change using JM smoothing
UC_FRC_JM = featureExtraction.RobustnessScore(UC_documents,code_documents,"JM",False)
code_FRC_JM = featureExtraction.RobustnessScore(code_documents,UC_documents,"JM",False)

print('UC_FRC_JM', UC_FRC_JM.shape)
print('code_FRC_JM', code_FRC_JM.shape)

# 2.5) First Rank Change using DP smoothing
UC_FRC_DP = featureExtraction.RobustnessScore(UC_documents,code_documents,"DP",False)
code_FRC_DP = featureExtraction.RobustnessScore(code_documents,UC_documents,"DP",False)

print('UC_FRC_DP', UC_FRC_DP.shape)
print('code_FRC_DP', code_FRC_DP.shape)

In [ ]:
# 4.1) Clustering Tendency using JensenShannon
UC_CT_JensenShannon = featureExtraction.ClusteringTendency(JS_UC.T,tfidf_matrix_uc,tfidf_matrix_code)
code_CT_JensenShannon = featureExtraction.ClusteringTendency(JS_CC,tfidf_matrix_code,tfidf_matrix_uc)

print("UC_CT_JensenShannon",UC_CT_JensenShannon.shape)
print("code_CT_JensenShannon",code_CT_JensenShannon.shape)

# 4.2) Clustering Tendency using VSM
UC_CT_VSM = featureExtraction.ClusteringTendency(cosine_similarity_UC.T,tfidf_matrix_uc,tfidf_matrix_code)
code_CT_VSM = featureExtraction.ClusteringTendency(cosine_similarity_CC,tfidf_matrix_code,tfidf_matrix_uc)

print("UC_CT_VSM",UC_CT_VSM.shape)
print("code_CT_VSM",code_CT_VSM.shape)

# 4.3) Clustering Tendency using BM25
UC_CT_BM25 = featureExtraction.ClusteringTendency(BM25_UC,tfidf_matrix_uc,tfidf_matrix_code)
code_CT_BM25 = featureExtraction.ClusteringTendency(BM25_CC,tfidf_matrix_code,tfidf_matrix_uc)

print("UC_CT_BM25",UC_CT_BM25.shape)
print("code_CT_BM25",code_CT_BM25.shape)

# 4.4) Clustering Tendency using JM
UC_CT_JM = featureExtraction.ClusteringTendency(JM_UC,tfidf_matrix_uc,tfidf_matrix_code)
code_CT_JM = featureExtraction.ClusteringTendency(JM_CC,tfidf_matrix_code,tfidf_matrix_uc)

print("UC_CT_JM",UC_CT_JM.shape)
print("code_CT_JM",code_CT_JM.shape)

# 4.5) Clustering Tendency using DP
UC_CT_DP = featureExtraction.ClusteringTendency(DP_UC,tfidf_matrix_uc,tfidf_matrix_code)
code_CT_DP = featureExtraction.ClusteringTendency(DP_CC,tfidf_matrix_code,tfidf_matrix_uc)

print("UC_CT_DP",UC_CT_DP.shape)
print("code_CT_DP",code_CT_DP.shape)


In [ ]:
# 5.1) Spatial AutoCorrelation using JensenShannon
UC_SAC_JS = featureExtraction.SpatialAutoCorrelation(JS_UC.T,tfidf_matrix_code)
code_SAC_JS = featureExtraction.SpatialAutoCorrelation(JS_CC,tfidf_matrix_uc)

print("UC_SAC_JS",UC_SAC_JS.shape)
print("code_SAC_JS",code_SAC_JS.shape)

# 5.2) Spatial AutoCorrelation using VSM
UC_SAC_VSM = featureExtraction.SpatialAutoCorrelation(cosine_similarity_UC.T,tfidf_matrix_code)
code_SAC_VSM = featureExtraction.SpatialAutoCorrelation(cosine_similarity_CC,tfidf_matrix_uc)

print("UC_SAC_VSM",UC_SAC_VSM.shape)
print("code_SAC_VSM",code_SAC_VSM.shape)

# 5.3) Spatial AutoCorrelation using BM25
UC_SAC_BM25 = featureExtraction.SpatialAutoCorrelation(BM25_UC,tfidf_matrix_code)
code_SAC_BM25 = featureExtraction.SpatialAutoCorrelation(BM25_CC,tfidf_matrix_uc)

print("UC_SAC_BM25",UC_SAC_BM25.shape)
print("code_SAC_BM25",code_SAC_BM25.shape)

# 5.4) Spatial AutoCorrelation using JM
UC_SAC_JM = featureExtraction.SpatialAutoCorrelation(JM_UC,tfidf_matrix_code)
code_SAC_JM = featureExtraction.SpatialAutoCorrelation(JM_CC,tfidf_matrix_uc)

print("UC_SAC_JM",UC_SAC_JM.shape)
print("code_SAC_JM",code_SAC_JM.shape)

# 5.5) Spatial AutoCorrelation using DP
UC_SAC_DP = featureExtraction.SpatialAutoCorrelation(DP_UC,tfidf_matrix_code)
code_SAC_DP = featureExtraction.SpatialAutoCorrelation(DP_CC,tfidf_matrix_uc)

print("UC_SAC_DP",UC_SAC_DP.shape)
print("code_SAC_DP",code_SAC_DP.shape)

In [ ]:
# 6.1) WIG using JensenShannon
UC_WIG_score_JensenShannon = featureExtraction.WeightedInformationGain(UC_documents,code_documents,JS_UC.T,tf_code_dict,np.sum(list(tf_code_dict.values())))
code_WIG_score_JensenShannon = featureExtraction.WeightedInformationGain(code_documents,UC_documents,JS_CC,tf_uc_dict,np.sum(list(tf_uc_dict.values())))

print("UC_WIG_score_JensenShannon",UC_WIG_score_JensenShannon.shape)
print("code_WIG_score_JensenShannon",code_WIG_score_JensenShannon.shape)

# 6.2) WIG Score using VSM 
UC_WIG_score_VSM = featureExtraction.WeightedInformationGain(UC_documents,code_documents,cosine_similarity_UC.T,tf_code_dict,np.sum(list(tf_code_dict.values())))
code_WIG_score_VSM = featureExtraction.WeightedInformationGain(code_documents,UC_documents,cosine_similarity_CC,tf_uc_dict,np.sum(list(tf_uc_dict.values())))

print("UC_WIG_score_VSM",UC_WIG_score_VSM.shape)
print("code_WIG_score_VSM",code_WIG_score_VSM.shape)

#6.3) WIG Score using BM25
UC_WIG_score_BM25 = featureExtraction.WeightedInformationGain(UC_documents,code_documents,BM25_UC,tf_code_dict,np.sum(list(tf_code_dict.values())))
code_WIG_score_BM25 = featureExtraction.WeightedInformationGain(code_documents,UC_documents,BM25_CC,tf_uc_dict,np.sum(list(tf_uc_dict.values())))

print("UC_WIG_score_BM25",UC_WIG_score_BM25.shape)
print("code_WIG_score_BM25",code_WIG_score_BM25.shape)

#6.4) WIG Score using JM
UC_WIG_score_JM = featureExtraction.WeightedInformationGain(UC_documents,code_documents,JM_UC,tf_code_dict,np.sum(list(tf_code_dict.values())))
code_WIG_score_JM = featureExtraction.WeightedInformationGain(code_documents,UC_documents,JM_CC,tf_uc_dict,np.sum(list(tf_uc_dict.values())))

print("UC_WIG_score_JM",UC_WIG_score_JM.shape)
print("code_WIG_score_JM",code_WIG_score_JM.shape)

#6.5) WIG Score using DP
UC_WIG_score_DP = featureExtraction.WeightedInformationGain(UC_documents,code_documents,DP_UC,tf_code_dict,np.sum(list(tf_code_dict.values())))
code_WIG_score_DP = featureExtraction.WeightedInformationGain(code_documents,UC_documents,DP_CC,tf_uc_dict,np.sum(list(tf_uc_dict.values())))

print("UC_WIG_score_DP",UC_WIG_score_DP.shape)
print("code_WIG_score_DP",code_WIG_score_DP.shape)


In [ ]:
# 7.1) NQC using JensenShannon
UC_NQC_JensenShannon = featureExtraction.NormalizedQueryCommitment(JS_UC.T)
code_NQC_JensenShannon = featureExtraction.NormalizedQueryCommitment(JS_CC)

print("UC_NQC_JensenShannon",UC_NQC_JensenShannon.shape)
print("code_NQC_JensenShannon",code_NQC_JensenShannon.shape)

# 7.2) NQC using VSM
UC_NQC_VSM = featureExtraction.NormalizedQueryCommitment(cosine_similarity_UC.T)
code_NQC_VSM = featureExtraction.NormalizedQueryCommitment(cosine_similarity_CC)

print("UC_NQC_VSM",UC_NQC_VSM.shape)
print("code_NQC_VSM",code_NQC_VSM.shape)

# 7.3) NQC using BM25
UC_NQC_BM25 = featureExtraction.NormalizedQueryCommitment(BM25_UC)
code_NQC_BM25 = featureExtraction.NormalizedQueryCommitment(BM25_CC)

print("UC_NQC_BM25",UC_NQC_BM25.shape)
print("code_NQC_BM25",code_NQC_BM25.shape)

# 7.4) NQC using JM
UC_NQC_JM = featureExtraction.NormalizedQueryCommitment(JM_UC)
code_NQC_JM = featureExtraction.NormalizedQueryCommitment(JM_CC)

print("UC_NQC_JM",UC_NQC_JM.shape)
print("code_NQC_JM",code_NQC_JM.shape)

# 7.5) NQC using DP
UC_NQC_DP = featureExtraction.NormalizedQueryCommitment(DP_UC)
code_NQC_DP = featureExtraction.NormalizedQueryCommitment(DP_CC)

print("UC_NQC_DP",UC_NQC_DP.shape)
print("code_NQC_DP",code_NQC_DP.shape)

### saving the post retrieval features into pickles

In [ ]:
np.save('./pickles/UC_queries_score_JensenShannon.npy', UC_queries_score_JensenShannon)
np.save('./pickles/code_queries_score_JensenShannon.npy', code_queries_score_JensenShannon)

np.save('./pickles/UC_queries_score_VSM.npy', UC_queries_score_VSM)
np.save('./pickles/code_queries_score_VSM.npy', code_queries_score_VSM)

np.save('./pickles/UC_queries_score_BM25.npy', UC_queries_score_BM25)
np.save('./pickles/code_queries_score_BM25.npy', code_queries_score_BM25)

np.save('./pickles/UC_queries_score_JM.npy', UC_queries_score_JM)
np.save('./pickles/code_queries_score_JM.npy', code_queries_score_JM)

np.save('./pickles/UC_queries_score_DP.npy', UC_queries_score_DP)
np.save('./pickles/code_queries_score_DP.npy', code_queries_score_DP)

In [ ]:
np.save('./pickles/UC_RS_JS.npy', UC_RS_JS)
np.save('./pickles/code_RS_JS.npy', code_RS_JS)

np.save('./pickles/UC_RS_VSM.npy', UC_RS_VSM)
np.save('./pickles/code_RS_VSM.npy', code_RS_VSM)

np.save('./pickles/UC_RS_BM25.npy', UC_RS_BM25)
np.save('./pickles/code_RS_BM25.npy', code_RS_BM25)

np.save('./pickles/UC_RS_JM.npy', UC_RS_JM)
np.save('./pickles/code_RS_JM.npy', code_RS_JM)

np.save('./pickles/UC_RS_DP.npy', UC_RS_DP)
np.save('./pickles/code_RS_DP.npy', code_RS_DP)

In [ ]:
np.save('./pickles/UC_FRC_JS.npy', UC_FRC_JS)
np.save('./pickles/code_FRC_JS.npy', code_FRC_JS)

np.save('./pickles/UC_FRC_VSM.npy', UC_FRC_VSM)
np.save('./pickles/code_FRC_VSM.npy', code_FRC_VSM)

np.save('./pickles/UC_FRC_BM25.npy', UC_FRC_BM25)
np.save('./pickles/code_FRC_BM25.npy', code_FRC_BM25)

np.save('./pickles/UC_FRC_JM.npy', UC_FRC_JM)
np.save('./pickles/code_FRC_JM.npy', code_FRC_JM)

np.save('./pickles/UC_FRC_DP.npy', UC_FRC_DP)
np.save('./pickles/code_FRC_DP.npy', code_FRC_DP)

In [ ]:
np.save('./pickles/UC_CT_JensenShannon.npy', UC_CT_JensenShannon)
np.save('./pickles/code_CT_JensenShannon.npy', code_CT_JensenShannon)

np.save('./pickles/UC_CT_VSM.npy', UC_CT_VSM)
np.save('./pickles/code_CT_VSM.npy', code_CT_VSM)

np.save('./pickles/UC_CT_BM25.npy', UC_CT_BM25)
np.save('./pickles/code_CT_BM25.npy', code_CT_BM25)

np.save('./pickles/UC_CT_JM.npy', UC_CT_JM)
np.save('./pickles/code_CT_JM.npy', code_CT_JM)

np.save('./pickles/UC_CT_DP.npy', UC_CT_DP)
np.save('./pickles/code_CT_DP.npy', code_CT_DP)

In [ ]:
np.save('./pickles/UC_SAC_JensenShannon.npy',UC_SAC_JS)
np.save('./pickles/code_SAC_JensenShannon.npy',code_SAC_JS)

np.save('./pickles/UC_SAC_VSM.npy',UC_SAC_VSM)
np.save('./pickles/code_SAC_VSM.npy',code_SAC_VSM)

np.save('./pickles/UC_SAC_BM25.npy',UC_SAC_BM25)
np.save('./pickles/code_SAC_BM25.npy',code_SAC_BM25)

np.save('./pickles/UC_SAC_JM.npy',UC_SAC_JM)
np.save('./pickles/code_SAC_JM.npy',code_SAC_JM)

np.save('./pickles/UC_SAC_DP.npy',UC_SAC_DP)
np.save('./pickles/code_SAC_DP.npy',code_SAC_DP)

In [ ]:
np.save('./pickles/UC_WIG_score_JensenShannon.npy',UC_WIG_score_JensenShannon)
np.save('./pickles/code_WIG_score_JensenShannon.npy',code_WIG_score_JensenShannon)

np.save('./pickles/UC_WIG_score_VSM.npy',UC_WIG_score_VSM)
np.save('./pickles/code_WIG_score_VSM.npy',code_WIG_score_VSM)

np.save('./pickles/UC_WIG_score_BM25.npy',UC_WIG_score_BM25)
np.save('./pickles/code_WIG_score_BM25.npy',code_WIG_score_BM25)

np.save('./pickles/UC_WIG_score_JM.npy',UC_WIG_score_JM)
np.save('./pickles/code_WIG_score_JM.npy',code_WIG_score_JM)

np.save('./pickles/UC_WIG_score_DP.npy',UC_WIG_score_DP)
np.save('./pickles/code_WIG_score_DP.npy',code_WIG_score_DP)

In [ ]:
np.save('./pickles/UC_NQC_JensenShannon.npy',UC_NQC_JensenShannon)
np.save('./pickles/code_NQC_JensenShannon.npy',code_NQC_JensenShannon)

np.save('./pickles/UC_NQC_VSM.npy',UC_NQC_VSM)
np.save('./pickles/code_NQC_VSM.npy',code_NQC_VSM)

np.save('./pickles/UC_NQC_BM25.npy',UC_NQC_BM25)
np.save('./pickles/code_NQC_BM25.npy',code_NQC_BM25)

np.save('./pickles/UC_NQC_JM.npy',UC_NQC_JM)
np.save('./pickles/code_NQC_JM.npy',code_NQC_JM)

np.save('./pickles/UC_NQC_DP.npy',UC_NQC_DP)
np.save('./pickles/code_NQC_DP.npy',code_NQC_DP)

### Document Statistics Features

In [12]:

num_terms_code, num_terms_UC, num_unique_terms_code, num_unique_terms_UC, num_overlapping_terms = featureExtraction.DocumentStatistics(UC_documents, code_documents)


### Loading the post retrieval features

In [10]:
UC_queries_score_JensenShannon = np.load('./pickles/UC_queries_score_JensenShannon.npy').reshape(-1, 1)
code_queries_score_JensenShannon = np.load('./pickles/code_queries_score_JensenShannon.npy').reshape(-1, 1)
print(code_queries_score_JensenShannon.shape)
normalizer.fit_transform(UC_queries_score_JensenShannon)
normalizer.fit_transform(code_queries_score_JensenShannon)

UC_queries_score_VSM = np.load('./pickles/UC_queries_score_VSM.npy').reshape(1, -1)
code_queries_score_VSM = np.load('./pickles/code_queries_score_VSM.npy').reshape(1, -1)

normalizer.fit_transform(UC_queries_score_VSM)
normalizer.fit_transform(code_queries_score_VSM)

UC_queries_score_BM25 = np.load('./pickles/UC_queries_score_BM25.npy').reshape(1, -1)
code_queries_score_BM25 = np.load('./pickles/code_queries_score_BM25.npy').reshape(1, -1)

normalizer.fit_transform(UC_queries_score_BM25)
normalizer.fit_transform(code_queries_score_BM25)

UC_queries_score_DP = np.load('./pickles/UC_queries_score_DP.npy').reshape(1, -1)
code_queries_score_DP = np.load('./pickles/code_queries_score_DP.npy').reshape(1, -1)

normalizer.fit_transform(UC_queries_score_DP)
normalizer.fit_transform(code_queries_score_DP)

UC_queries_score_JM = np.load('./pickles/UC_queries_score_JM.npy').reshape(1, -1)
code_queries_score_JM = np.load('./pickles/code_queries_score_JM.npy').reshape(1, -1)

normalizer.fit_transform(UC_queries_score_JM)
normalizer.fit_transform(code_queries_score_JM)

UC_RS_JS = np.load('./pickles/UC_RS_JS.npy').reshape(1, -1)
code_RS_JS = np.load('./pickles/code_RS_JS.npy').reshape(1, -1)

normalizer.fit_transform(UC_RS_JS)
normalizer.fit_transform(code_RS_JS)

UC_RS_VSM = np.load('./pickles/UC_RS_VSM.npy').reshape(1, -1)
code_RS_VSM = np.load('./pickles/code_RS_VSM.npy').reshape(1, -1)

normalizer.fit_transform(UC_RS_VSM)
normalizer.fit_transform(code_RS_VSM)

UC_RS_BM25 = np.load('./pickles/UC_RS_BM25.npy').reshape(1, -1)
code_RS_BM25 = np.load('./pickles/code_RS_BM25.npy').reshape(1, -1)

normalizer.fit_transform(UC_RS_BM25)
normalizer.fit_transform(code_RS_BM25)

UC_RS_JM = np.load('./pickles/UC_RS_JM.npy').reshape(1, -1)
code_RS_JM = np.load('./pickles/code_RS_JM.npy').reshape(1, -1)

normalizer.fit_transform(UC_RS_JM)
normalizer.fit_transform(code_RS_JM)

UC_RS_DP = np.load('./pickles/UC_RS_DP.npy').reshape(1, -1)
code_RS_DP = np.load('./pickles/code_RS_DP.npy').reshape(1, -1)

normalizer.fit_transform(UC_RS_DP)
normalizer.fit_transform(code_RS_DP)

UC_FRC_JS = np.load('./pickles/UC_FRC_JS.npy').reshape(1, -1)
code_FRC_JS = np.load('./pickles/code_FRC_JS.npy').reshape(1, -1)

normalizer.fit_transform(UC_FRC_JS)
normalizer.fit_transform(code_FRC_JS)

UC_FRC_VSM = np.load('./pickles/UC_FRC_VSM.npy').reshape(1, -1)
code_FRC_VSM = np.load('./pickles/code_FRC_VSM.npy').reshape(1, -1)

normalizer.fit_transform(UC_FRC_VSM)
normalizer.fit_transform(code_FRC_VSM)

UC_FRC_BM25 = np.load('./pickles/UC_FRC_BM25.npy').reshape(1, -1)
code_FRC_BM25 = np.load('./pickles/code_FRC_BM25.npy').reshape(1, -1)

normalizer.fit_transform(UC_FRC_BM25)
normalizer.fit_transform(code_FRC_BM25)

UC_FRC_JM = np.load('./pickles/UC_FRC_JM.npy').reshape(1, -1)
code_FRC_JM = np.load('./pickles/code_FRC_JM.npy').reshape(1, -1)

normalizer.fit_transform(UC_FRC_JM)
normalizer.fit_transform(code_FRC_JM)

UC_FRC_DP = np.load('./pickles/UC_FRC_DP.npy').reshape(1, -1)
code_FRC_DP = np.load('./pickles/code_FRC_DP.npy').reshape(1, -1)

normalizer.fit_transform(UC_FRC_DP)
normalizer.fit_transform(code_FRC_DP)

UC_CT_JensenShannon = np.load('./pickles/UC_CT_JensenShannon.npy').reshape(1, -1)
code_CT_JensenShannon = np.load('./pickles/code_CT_JensenShannon.npy').reshape(1, -1)

normalizer.fit_transform(UC_CT_JensenShannon)
normalizer.fit_transform(code_CT_JensenShannon)

UC_CT_VSM = np.load('./pickles/UC_CT_VSM.npy').reshape(1, -1)
code_CT_VSM = np.load('./pickles/code_CT_VSM.npy').reshape(1, -1)

normalizer.fit_transform(UC_CT_VSM)
normalizer.fit_transform(code_CT_VSM)

UC_CT_BM25 = np.load('./pickles/UC_CT_BM25.npy').reshape(1, -1)
code_CT_BM25 = np.load('./pickles/code_CT_BM25.npy').reshape(1, -1)

normalizer.fit_transform(UC_CT_BM25)
normalizer.fit_transform(code_CT_BM25)

UC_CT_JM = np.load('./pickles/UC_CT_JM.npy').reshape(1, -1)
code_CT_JM = np.load('./pickles/code_CT_JM.npy').reshape(1, -1)

normalizer.fit_transform(UC_CT_JM)
normalizer.fit_transform(code_CT_JM)

UC_CT_DP = np.load('./pickles/UC_CT_DP.npy').reshape(1, -1)
code_CT_DP = np.load('./pickles/code_CT_DP.npy').reshape(1, -1)

normalizer.fit_transform(UC_CT_DP)
normalizer.fit_transform(code_CT_DP)

UC_SAC_JS = np.load('./pickles/UC_SAC_JensenShannon.npy')
code_SAC_JS = np.load('./pickles/code_SAC_JensenShannon.npy')

normalizer.fit_transform(UC_SAC_JS)
normalizer.fit_transform(code_SAC_JS)

UC_SAC_VSM = np.load('./pickles/UC_SAC_VSM.npy')
code_SAC_VSM = np.load('./pickles/code_SAC_VSM.npy')

normalizer.fit_transform(UC_SAC_VSM)
normalizer.fit_transform(code_SAC_VSM)

UC_SAC_BM25 = np.load('./pickles/UC_SAC_BM25.npy')
code_SAC_BM25 = np.load('./pickles/code_SAC_BM25.npy')

normalizer.fit_transform(UC_SAC_BM25)
normalizer.fit_transform(code_SAC_BM25)

UC_SAC_JM = np.load('./pickles/UC_SAC_JM.npy')
code_SAC_JM = np.load('./pickles/code_SAC_JM.npy')

normalizer.fit_transform(UC_SAC_JM)
normalizer.fit_transform(code_SAC_JM)

UC_SAC_DP = np.load('./pickles/UC_SAC_DP.npy')
code_SAC_DP = np.load('./pickles/code_SAC_DP.npy')

normalizer.fit_transform(UC_SAC_DP)
normalizer.fit_transform(code_SAC_DP)

UC_WIG_score_JensenShannon = np.load('./pickles/UC_WIG_score_JensenShannon.npy')
code_WIG_score_JensenShannon = np.load('./pickles/code_WIG_score_JensenShannon.npy')

normalizer.fit_transform(UC_WIG_score_JensenShannon)
normalizer.fit_transform(code_WIG_score_JensenShannon)

UC_WIG_score_VSM = np.load('./pickles/UC_WIG_score_VSM.npy')
code_WIG_score_VSM = np.load('./pickles/code_WIG_score_VSM.npy')

normalizer.fit_transform(UC_WIG_score_VSM)
normalizer.fit_transform(code_WIG_score_VSM)

UC_WIG_score_BM25 = np.load('./pickles/UC_WIG_score_BM25.npy')
code_WIG_score_BM25 = np.load('./pickles/code_WIG_score_BM25.npy')

normalizer.fit_transform(UC_WIG_score_BM25)
normalizer.fit_transform(code_WIG_score_BM25)

UC_WIG_score_JM = np.load('./pickles/UC_WIG_score_JM.npy')
code_WIG_score_JM = np.load('./pickles/code_WIG_score_JM.npy')

normalizer.fit_transform(UC_WIG_score_JM)
normalizer.fit_transform(code_WIG_score_JM)

UC_WIG_score_DP = np.load('./pickles/UC_WIG_score_DP.npy')
code_WIG_score_DP = np.load('./pickles/code_WIG_score_DP.npy')

normalizer.fit_transform(UC_WIG_score_DP)
normalizer.fit_transform(code_WIG_score_DP)

UC_NQC_JensenShannon = np.load('./pickles/UC_NQC_JensenShannon.npy')
code_NQC_JensenShannon = np.load('./pickles/code_NQC_JensenShannon.npy')

normalizer.fit_transform(UC_NQC_JensenShannon)
normalizer.fit_transform(code_NQC_JensenShannon)

UC_NQC_VSM = np.load('./pickles/UC_NQC_VSM.npy')
code_NQC_VSM = np.load('./pickles/code_NQC_VSM.npy')

normalizer.fit_transform(UC_NQC_VSM)
normalizer.fit_transform(code_NQC_VSM)

UC_NQC_BM25 = np.load('./pickles/UC_NQC_BM25.npy')
code_NQC_BM25 = np.load('./pickles/code_NQC_BM25.npy')

normalizer.fit_transform(UC_NQC_BM25)
normalizer.fit_transform(code_NQC_BM25)

UC_NQC_JM = np.load('./pickles/UC_NQC_JM.npy')
code_NQC_JM = np.load('./pickles/code_NQC_JM.npy')

normalizer.fit_transform(UC_NQC_JM)
normalizer.fit_transform(code_NQC_JM)

UC_NQC_DP = np.load('./pickles/UC_NQC_DP.npy')
code_NQC_DP = np.load('./pickles/code_NQC_DP.npy')

normalizer.fit_transform(UC_NQC_DP)
normalizer.fit_transform(code_NQC_DP)

(119, 1)


array([[0.37103327],
       [0.37103327],
       [0.37103327],
       [0.90915987],
       [0.45877988],
       [0.65839807],
       [0.63354976],
       [0.47736004],
       [0.89337862],
       [0.15234764],
       [0.5048334 ],
       [0.57958497],
       [0.90689592],
       [0.51877333],
       [0.51877333],
       [0.43878994],
       [0.59612053],
       [0.42562225],
       [0.47396338],
       [0.37103327],
       [0.37103327],
       [0.61846342],
       [0.37103327],
       [0.37103327],
       [0.57958497],
       [0.37103327],
       [0.        ],
       [0.51877333],
       [0.37103327],
       [0.37103327],
       [0.42398511],
       [0.44210161],
       [0.37103327],
       [0.37103327],
       [0.37103327],
       [0.37103327],
       [0.37103327],
       [0.37103327],
       [0.37103327],
       [0.38686232],
       [0.37103327],
       [0.37103327],
       [0.44210161],
       [0.37103327],
       [0.37103327],
       [0.47736004],
       [0.37103327],
       [0.438

In [92]:
###### Remove hardcoded sizing with the size captured from the data set.
avg_idf_uc_reshaped = np.tile(avg_idf_uc, (119,1))
avg_idf_code_reshaped = np.tile(avg_idf_code, (58,1))

max_idf_uc_reshaped = np.tile(max_idf_uc, (119,1))
max_idf_code_reshaped = np.tile(max_idf_code, (58,1))

dev_idf_uc_reshaped = np.tile(dev_idf_uc, (119,1))
dev_idf_code_reshaped = np.tile(dev_idf_code, (58,1))

avg_ictf_uc_reshaped = np.tile(avg_ictf_uc, (119,1))
avg_ictf_code_reshaped = np.tile(avg_ictf_code, (58,1))

max_ictf_uc_reshaped = np.tile(max_ictf_uc, (119,1))
max_ictf_code_reshaped = np.tile(max_ictf_code, (58,1))

dev_ictf_uc_reshaped = np.tile(dev_ictf_uc, (119,1))
dev_ictf_code_reshaped = np.tile(dev_ictf_code, (58,1))

avg_entropy_uc_reshaped = np.tile(avg_entropy_uc, (119,1))
avg_entropy_code_reshaped = np.tile(avg_entropy_code, (58,1))

max_entropy_uc_reshaped = np.tile(max_entropy_uc, (119,1)) # ------> gets removed, 2nd round
max_entropy_code_reshaped = np.tile(max_entropy_code, (58,1)) # ------> gets removed, 2nd round

med_entropy_uc_reshaped = np.tile(med_entropy_uc, (119,1))
med_entropy_code_reshaped = np.tile(med_entropy_code, (58,1))

dev_entropy_uc_reshaped = np.tile(dev_entropy_uc, (119,1))
dev_entropy_code_reshaped = np.tile(dev_entropy_code, (58,1)) # -------> gets removed

avg_variance_uc_reshaped = np.tile(avg_variance_uc, (119,1))
avg_variance_code_reshaped = np.tile(avg_variance_code, (58,1))

max_variance_uc_reshaped = np.tile(max_variance_uc, (119,1))
max_variance_code_reshaped = np.tile(max_variance_code, (58,1))

sum_variance_uc_reshaped = np.tile(sum_variance_uc, (119,1)) # --------> gets removed
sum_variance_code_reshaped = np.tile(sum_variance_code, (58,1)) # -------> gets removed

avg_scq_uc_reshaped = np.tile(avg_scq_uc, (119,1))
avg_scq_code_reshaped = np.tile(avg_scq_code, (58,1)) # -------> gets removed

max_scq_uc_reshaped = np.tile(max_scq_uc, (119,1))
max_scq_code_reshaped = np.tile(max_scq_code, (58,1))

sum_sqc_uc_reshaped = np.tile(sum_sqc_uc, (119,1))# --------> gets removed, 2nd round
sum_sqc_code_reshaped = np.tile(sum_sqc_code, (58,1)) # --------> gets removed, 2nd round

avg_pmi_uc_reshaped = np.tile(avg_pmi_uc, (119,1)) # --------> gets removed 
avg_pmi_code_reshaped = np.tile(avg_pmi_code, (58,1)) # --------> gets removed 

max_pmi_uc_reshaped = np.tile(max_pmi_uc, (119,1))
max_pmi_code_reshaped = np.tile(max_pmi_code, (58,1))

qs_uc_reshaped = np.tile(qs_uc, (119,1)) # --------> gets removed, 2nd round
qs_code_reshaped = np.tile(qs_code, (58,1)) # --------> gets removed, 2nd round


UC_queries_score_JensenShannon_reshaped = np.tile(UC_queries_score_JensenShannon, (1,119))
code_queries_score_JensenShannon_reshaped = np.tile(code_queries_score_JensenShannon, (1,58))

UC_queries_score_VSM_reshaped =  np.tile(UC_queries_score_VSM, (119,1))
code_queries_score_VSM_reshaped = np.tile(code_queries_score_VSM, (58,1))

UC_queries_score_BM25_reshaped = np.tile(UC_queries_score_BM25, (119,1))
code_queries_score_BM25_reshaped = np.tile(code_queries_score_BM25, (58,1))

UC_queries_score_JM_reshaped = np.tile(UC_queries_score_JM, (119,1))
code_queries_score_JM_reshaped = np.tile(code_queries_score_JM, (58,1))

UC_queries_score_JM_reshaped = np.tile(UC_queries_score_JM, (119,1))
code_queries_score_JM_reshaped = np.tile(code_queries_score_JM, (58,1))

UC_queries_score_DP_reshaped = np.tile(UC_queries_score_DP, (119,1))
code_queries_score_DP_reshaped = np.tile(code_queries_score_DP, (58,1))

UC_RS_JS_reshaped = np.tile(UC_RS_JS, (119,1))
code_RS_JS_reshaped = np.tile(code_RS_JS, (58,1))

UC_RS_VSM_reshaped = np.tile(UC_RS_VSM, (119,1))
code_RS_VSM_reshaped = np.tile(code_RS_VSM, (58,1))

UC_RS_BM25_reshaped = np.tile(UC_RS_BM25, (119,1))
code_RS_BM25_reshaped = np.tile(code_RS_BM25, (58,1))

UC_RS_JM_reshaped = np.tile(UC_RS_JM, (119,1))
code_RS_JM_reshaped = np.tile(code_RS_JM, (58,1))

UC_RS_DP_reshaped = np.tile(UC_RS_DP,(119,1))
code_RS_DP_reshaped = np.tile(code_RS_DP, (58,1))

UC_FRC_JS_reshaped = np.tile(UC_FRC_JS,(119,1))
code_FRC_JS_reshaped = np.tile(code_FRC_JS, (58,1))

UC_FRC_VSM_reshaped = np.tile(UC_FRC_VSM, (119,1))
code_FRC_VSM_reshaped = np.tile(code_FRC_VSM, (58,1))

UC_FRC_BM25_reshaped = np.tile(UC_FRC_BM25, (119,1))
code_FRC_BM25_reshaped = np.tile(code_FRC_BM25, (58,1))

UC_FRC_JM_reshaped = np.tile(UC_FRC_JM, (119,1))
code_FRC_JM_reshaped = np.tile(code_FRC_JM, (58,1))

UC_FRC_DP_reshaped = np.tile(UC_FRC_DP, (119,1))
code_FRC_DP_reshaped = np.tile(code_FRC_DP, (58,1))

UC_CT_JensenShannon_reshaped = np.tile(UC_CT_JensenShannon, (119,1))
code_CT_JensenShannon_reshaped = np.tile(code_CT_JensenShannon, (58,1))

UC_CT_VSM_reshaped = np.tile(UC_CT_VSM, (119,1))
code_CT_VSM_reshaped = np.tile(code_CT_VSM, (58,1))

UC_CT_BM25_reshaped = np.tile(UC_CT_BM25, (119,1))
code_CT_BM25_reshaped = np.tile(code_CT_BM25, (58,1))

UC_CT_JM_reshaped = np.tile(UC_CT_JM, (119,1))
code_CT_JM_reshaped = np.tile(code_CT_JM, (58,1))

UC_CT_DP_reshaped = np.tile(UC_CT_DP, (119,1))
code_CT_DP_reshaped = np.tile(code_CT_DP, (58,1))

UC_SAC_JS_reshaped = np.tile(UC_SAC_JS, (1,119))
code_SAC_JS_reshaped = np.tile(code_SAC_JS, (1,58))

UC_SAC_VSM_reshaped = np.tile(UC_SAC_VSM, (1,119))
code_SAC_VSM_reshaped = np.tile(code_SAC_VSM, (1,58))

UC_SAC_BM25_reshaped = np.tile(UC_SAC_BM25, (1,119))
code_SAC_BM25_reshaped = np.tile(code_SAC_BM25, (1,58))

UC_SAC_JM_reshaped = np.tile(UC_SAC_JM, (1,119))
code_SAC_JM_reshaped = np.tile(code_SAC_JM, (1,58))

UC_SAC_DP_reshaped = np.tile(UC_SAC_DP, (1,119))
code_SAC_DP_reshaped = np.tile(code_SAC_DP, (1,58))

UC_WIG_score_JensenShannon_reshaped = np.tile(UC_WIG_score_JensenShannon, (1,119))
code_WIG_score_JensenShannon_reshaped = np.tile(code_WIG_score_JensenShannon, (1,58))

UC_WIG_score_VSM_reshaped = np.tile(UC_WIG_score_VSM, (1,119))
code_WIG_score_VSM_reshaped = np.tile(code_WIG_score_VSM, (1,58))

UC_WIG_score_BM25_reshaped = np.tile(UC_WIG_score_BM25, (1,119))
code_WIG_score_BM25_reshaped = np.tile(code_WIG_score_BM25, (1,58))

UC_WIG_score_JM_reshaped = np.tile(UC_WIG_score_JM, (1,119))
code_WIG_score_JM_reshaped = np.tile(code_WIG_score_JM, (1,58))

UC_WIG_score_DP_reshaped = np.tile(UC_WIG_score_DP, (1,119))
code_WIG_score_DP_reshaped = np.tile(code_WIG_score_DP, (1,58))

UC_NQC_JensenShannon_reshaped = np.tile(UC_NQC_JensenShannon, (1,119))
code_NQC_JensenShannon_reshaped = np.tile(code_NQC_JensenShannon, (1,58))

UC_NQC_VSM_reshaped = np.tile(UC_NQC_VSM, (1,119))
code_NQC_VSM_reshaped = np.tile(code_NQC_VSM, (1,58))

UC_NQC_BM25_reshaped = np.tile(UC_NQC_BM25, (1,119))
code_NQC_BM25_reshaped = np.tile(code_NQC_BM25, (1,58))

UC_NQC_JM_reshaped = np.tile(UC_NQC_JM, (1,119))
code_NQC_JM_reshaped = np.tile(code_NQC_JM, (1,58))

UC_NQC_DP_reshaped = np.tile(UC_NQC_DP, (1,119))
code_NQC_DP_reshaped = np.tile(code_NQC_DP, (1,58))

UC_SCS_reshaped = np.tile(UC_SCS, (119,1))
CC_SCS_reshaped = np.tile(CC_SCS, (58,1))

UC_CoherenceScore_reshaped = np.tile(UC_CoherenceScore, (119,1))
CC_CoherenceScore_reshaped = np.tile(CC_CoherenceScore, (58,1))

num_terms_UC_reshaped = np.tile(num_terms_UC, (119,1))
num_terms_code_reshaped = np.tile(num_terms_code, (58,1))

num_unique_terms_UC_reshaped = np.tile(num_unique_terms_UC, (119,1))
num_unique_terms_code_reshaped = np.tile(num_unique_terms_code, (58,1))


feature_matrix = np.stack(( cosine_similarity_UC, cosine_similarity_CC, LSA_similraities_UC, LSA_similraities_CC,
                            JS_UC, JS_CC,LDA_similraities_UC,LDA_similraities_CC,BM25_UC.T,BM25_CC,JM_UC.T,JM_CC,DP_UC.T,DP_CC,
                            avg_idf_uc_reshaped.T, avg_idf_code_reshaped, max_idf_uc_reshaped.T,
                            max_idf_code_reshaped, dev_idf_uc_reshaped.T, dev_idf_code_reshaped, avg_ictf_uc_reshaped.T,
                            avg_ictf_code_reshaped, max_ictf_uc_reshaped.T, max_ictf_code_reshaped, dev_ictf_uc_reshaped.T,
                            dev_ictf_code_reshaped, avg_entropy_uc_reshaped.T, avg_entropy_code_reshaped, max_entropy_uc_reshaped.T,
                            max_entropy_code_reshaped, med_entropy_uc_reshaped.T, med_entropy_code_reshaped, dev_entropy_uc_reshaped.T, 
                            dev_entropy_code_reshaped, avg_variance_uc_reshaped.T, avg_variance_code_reshaped, max_variance_uc_reshaped.T,
                            max_variance_code_reshaped, sum_variance_uc_reshaped.T, sum_variance_code_reshaped, avg_scq_uc_reshaped.T,
                            avg_scq_code_reshaped, max_scq_uc_reshaped.T, max_scq_code_reshaped, sum_sqc_uc_reshaped.T, sum_sqc_code_reshaped,
                            avg_pmi_uc_reshaped.T, avg_pmi_code_reshaped, max_pmi_uc_reshaped.T, max_pmi_code_reshaped, qs_uc_reshaped.T, qs_code_reshaped, 
                            
                            UC_queries_score_JensenShannon_reshaped, code_queries_score_JensenShannon_reshaped.T,
                            UC_queries_score_VSM_reshaped.T, code_queries_score_VSM_reshaped,
                            UC_queries_score_BM25_reshaped.T, code_queries_score_BM25_reshaped,
                            UC_queries_score_JM_reshaped.T, code_queries_score_JM_reshaped,
                            UC_queries_score_DP_reshaped.T, code_queries_score_DP_reshaped,
                             UC_RS_JS_reshaped.T, code_RS_JS_reshaped, UC_RS_VSM_reshaped.T, 
                             code_RS_VSM_reshaped,UC_RS_BM25_reshaped.T,code_RS_BM25_reshaped,UC_RS_JM_reshaped.T,
                             code_RS_JM_reshaped,UC_RS_DP_reshaped.T,code_RS_DP_reshaped,
                             UC_FRC_JS_reshaped.T,code_FRC_JS_reshaped,
                            UC_FRC_VSM_reshaped.T,code_FRC_VSM_reshaped,UC_FRC_BM25_reshaped.T,code_FRC_BM25_reshaped,UC_FRC_JM_reshaped.T,code_FRC_JM_reshaped,UC_FRC_DP_reshaped.T,code_FRC_DP_reshaped,UC_CT_JensenShannon_reshaped.T,code_CT_JensenShannon_reshaped,
                            UC_CT_VSM_reshaped.T,code_CT_VSM_reshaped,UC_CT_BM25_reshaped.T,code_CT_BM25_reshaped,UC_CT_JM_reshaped.T,code_CT_JM_reshaped,UC_CT_DP_reshaped.T,code_CT_DP_reshaped,UC_SAC_JS_reshaped,code_SAC_JS_reshaped.T,UC_SAC_VSM_reshaped,code_SAC_VSM_reshaped.T,UC_SAC_BM25_reshaped,
                            code_SAC_BM25_reshaped.T,UC_SAC_JM_reshaped,code_SAC_JM_reshaped.T,UC_SAC_DP_reshaped,code_SAC_DP_reshaped.T,UC_WIG_score_JensenShannon_reshaped,code_WIG_score_JensenShannon_reshaped.T,UC_WIG_score_VSM_reshaped,
                            code_WIG_score_VSM_reshaped.T,UC_WIG_score_BM25_reshaped,code_WIG_score_BM25_reshaped.T,UC_WIG_score_JM_reshaped,code_WIG_score_JM_reshaped.T,UC_WIG_score_DP_reshaped,code_WIG_score_DP_reshaped.T,UC_NQC_JensenShannon_reshaped,
                            code_NQC_JensenShannon_reshaped.T,
                            UC_NQC_VSM_reshaped,code_NQC_VSM_reshaped.T
                            ,UC_NQC_BM25_reshaped,code_NQC_BM25_reshaped.T,
                            UC_NQC_JM_reshaped,code_NQC_JM_reshaped.T,
                            UC_NQC_DP_reshaped,code_NQC_DP_reshaped.T,
                            UC_CoherenceScore_reshaped.T,CC_CoherenceScore_reshaped,CC_SCS_reshaped,UC_SCS_reshaped.T,
                            num_overlapping_terms,num_terms_code_reshaped, num_terms_UC_reshaped.T,
                            num_unique_terms_code_reshaped, num_unique_terms_UC_reshaped.T
                            ), axis=2)

print(feature_matrix.shape)


(58, 119, 131)


### Document Statistics Features

In [93]:
num_terms_code =[]
for document in code_documents:
    num_terms_code.append(len(document.split()))

num_terms_code=np.array(num_terms_code)
print(num_terms_code)

num_terms_UC =[]
for document in UC_documents:
    num_terms_UC.append(len(document.split()))

num_terms_UC=np.array(num_terms_UC)
print(num_terms_UC)

num_unique_terms_code =[]
for document in code_documents:
    num_unique_terms_code.append(len(set(document.split())))

num_unique_terms_code=np.array(num_unique_terms_code)
print(num_unique_terms_code)

num_unique_terms_UC =[]
for document in UC_documents:
    num_unique_terms_UC.append(len(set(document.split())))

num_unique_terms_UC=np.array(num_unique_terms_UC)
print(num_unique_terms_UC)

# each column represent D1 intersect with all usecases, then column2= D2 intersect with all usescases....
#if want to access the intesection between a specific usecase and all coressponding documents you will access the row index coressponding to this usecase 
num_overlapping_terms_code =np.zeros((len(UC_documents),len(code_documents)))
for uc_index,uc in enumerate(UC_documents):
    for code_index,code in enumerate(code_documents):
        num_overlapping_terms_code[uc_index][code_index] = 100*(len(set(uc.split()).intersection(set(code.split()))))/(num_unique_terms_UC[uc_index]+num_unique_terms_code[code_index])

print(num_overlapping_terms_code)




[ 151 2366  420  187  117  161  637  310  152  227  239  298  167  101
  219  675  128  637  258  254 2863  204  404  858  201  168  182   76
  246  254  208  277  254  409  724  205  301  494  409  487 2208  974
  291  182  160  237  844 1377  441   98  781 1534  629  407   51  636
  629 1623  766  185  432 2319  301  720  248   84  166  595  183  160
  160   96  124  124  374  122  584  291  372  179  168  170  122   35
  294  305  355  237   67  148  259  258  272   89  246   98 2818  208
  211  194  173 1735  246  182 2266  273  973   75  133 3864 4576 1807
  167  184 1194  208  298 2919  731]
[54 40 41 38 39 39 66 39 81 53 59 66 93 58 45 72 37 51 77 38 45 35 80 56
 44 30 56 39 49 34 33 89 84 44 61 81 58 72 69 40 57 63 54 34 38 39 42 46
 89 52 38 40 56 46 45 51 87 42]
[ 51 270 126  56  46  41  61  46  40  42  44  45  41  38  42  59  38  62
  45  44 308  62  99 127  67  60  68  51  54  53  84  64  53  75  81  61
  71  95  75  78 146  88  74  35  45  39 106 167  80  47 131 195 110  9

## Feature Selection

In [94]:
# matrix shape = links * features 
# ----features------
#|
#links
#|
feature_matrix_reshaped = feature_matrix.reshape(feature_matrix.shape[0]*feature_matrix.shape[1], -1)
print(feature_matrix_reshaped.shape)
correlation_features=np.corrcoef(feature_matrix_reshaped,rowvar=False)
print(correlation_features.shape)
features_excluded=set()

for i in range(correlation_features.shape[1]):
    for j in range(i+1,correlation_features.shape[0]):
        if (correlation_features[j][i] >= 0.9 ):
            print(j,i)
            features_excluded.add(j)

features_links_selected=np.delete(feature_matrix_reshaped, list(features_excluded), axis=1) 
print(features_excluded)
print(features_links_selected.shape)
features_links_selected_reshaped = features_links_selected.reshape(feature_matrix.shape[0], feature_matrix.shape[1], -1)
print(features_links_selected_reshaped.shape)
# correlation_features)
# for i in correlation_features:
#     print (i)


(6902, 131)
(131, 131)
10 8
11 9
128 13
130 13
111 103
109 107
111 107
110 108
111 109
116 114
118 114
117 115
119 115
118 116
119 117
129 127
130 128
{128, 129, 130, 10, 11, 109, 110, 111, 116, 117, 118, 119}
(6902, 119)
(58, 119, 119)


c:\Users\basse\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\basse\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


### Mapping Features to Labels

In [95]:
Features = list()
for row in DataSet.index:
    index_code = int(DataSet.loc[row, 'CC'])
    index_UC = int(DataSet.loc[row, 'UC'])
    Features.append(features_links_selected_reshaped[index_UC][index_code])


## Data Imbalancing

In [96]:
from imblearn.over_sampling import SMOTE 
smote = SMOTE(random_state=42)
Features_SMOTE, Labels_SMOTE = smote.fit_resample(Features, DataSet['Labels'].to_list() )

## Model

In [97]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

train_features, test_features, train_labels, test_labels = train_test_split(Features_SMOTE, Labels_SMOTE, test_size = 0.25, random_state = 42)
model_random_forest = RandomForestRegressor(n_estimators = 1000, random_state = 42)
model_random_forest.fit(train_features, train_labels)

predictions = model_random_forest.predict(test_features)
errors = abs(predictions - test_labels)
print('mean Absolute Error:', round(np.mean(errors), 2) )


KeyboardInterrupt: 

In [28]:
acc=(test_labels==predictions).sum()
print(acc/len(test_labels))

0.2575068243858053


In [ ]:
import csv
with open('answerSet.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    valid_links_labels = []
    for row in reader:
        temp=row[0].split(",")
        #match = re.search(rprint('UC(\d+)\.txt', temp[0])
        valid_links_labels.append((temp[0],temp[1]))
        # if (valid_links_labels.get(temp[1])==None):
        # valid_links_labels[temp[1]]=[int(match.group(1))]
        # else:
        #     valid_links_labels[temp[1]].append(int(match.group(1)))

# file_names = list(valid_links_labels.keys())
# file_names.sort()
# valid_links_labels_sorted = {i: valid_links_labels[i] for i in file_names}
            
# valid_links_labels_sorted)
# len(valid_links_labels_sorted.keys()))
print(valid_links_labels)